<table>
<tr>    
<td style="text-align: center">
<h1>Machine Learning for Disease Prediction - Working with Tabular Data</h1>
<h2><a href="http://home.agh.edu.pl/~horzyk/index.php">Adrian Horzyk</a></h2>
</td> 
<td>
<img src="http://home.agh.edu.pl/~horzyk/im/AdrianHorzyk49BT140h.png" alt="Adrian Horzyk, Professor" title="Adrian Horzyk, Professor" />        
</td> 
</tr>
</table>
<h3><i>Welcome to the interactive notebook where you can learn how neural networks work, experience and check their operation on selected data sets, and conduct your own experiments.</i></h3>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import subprocess
 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from pytorch_tabnet.tab_model import TabNetClassifier
import shap

/opt/homebrew/anaconda3/envs/biomedicine/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Function to load different biomedical datasets
def load_dataset(dataset_name):
    datasets = {
        "diabetes": {
            "url": "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv",
            "columns": ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
        },
        "heart_disease": {
            "url": "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/heart/heart.csv",
            "columns": None  # Column names are included in the dataset
        },
        "breast_cancer": {
            "url": "https://raw.githubusercontent.com/dataprofessor/data/master/breast_cancer.csv",
            "columns": None  # Column names are included in the dataset
        }
    }
    
    if dataset_name in datasets:
        url = datasets[dataset_name]["url"]
        columns = datasets[dataset_name]["columns"]
        df = pd.read_csv(url, names=columns) if columns else pd.read_csv(url)
        return df
    else:
        raise ValueError("Dataset not found. Available options: 'diabetes', 'heart_disease', 'breast_cancer'")

## Here, we choose "diabetes" dataset out of the above three for the following experiments.
### After making yourself familiar with this notebook, use also the other two datasets for similar experiments for training your skills and understanding of this topic.

In [3]:
# Load dataset
selected_dataset = "diabetes"  # Change this to 'heart_disease' or 'breast_cancer' to try other datasets
df = load_dataset(selected_dataset)

In [4]:
# Handle missing values using mean imputation for numerical columns
imputer = SimpleImputer(strategy='mean')  # Can change to 'median' or 'most_frequent'
df.iloc[:, :-1] = imputer.fit_transform(df.iloc[:, :-1])

In [5]:
# Train-test split (80% training, 20% testing)
X = df.drop(columns=[df.columns[-1]])  # Last column is assumed to be the target
y = df[df.columns[-1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Feature Scaling using StandardScaler for better performance in distance-based models
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Exercise: Experiment with different hyperparameters
Try modifying n_estimators, max_depth, learning_rate, kernel types, and other settings to see how performance changes.
Conduct some experiments with the hyperparameters and try to achieve better results.

In [ ]:
# Dictionary of machine learning models with key hyperparameters
models = {
    "RandomForest": RandomForestClassifier(
        n_estimators=100,
        max_depth=5,
        random_state=42
    ),
    "SVM": SVC(
        kernel='rbf',
        C=1,
        gamma='scale',
        probability=True
    ),
    "XGBoost": XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        scale_pos_weight=10
    ),
    "ANN": Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ]),
    "DNN": Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
}

: 

In [ ]:
# Compile and fit ANN & DNN models
for model_name in ["ANN", "DNN"]:
    models[model_name].compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    models[model_name].fit(X_train, y_train, epochs=50, batch_size=32,verbose=1)

In [ ]:
# Training and Evaluation of ML models
results = {}
y_prob_dict = {}
for name, model in models.items():
    if name in ["ANN", "DNN"]:
        y_prob = model.predict(X_test).flatten()  # Keep probabilities for AUC-ROC
        y_pred = (y_prob > 0.5).astype(int)  # Convert probabilities to binary labels
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:,1] if hasattr(model, 'predict_proba') else None
    
    y_prob_dict[name] = y_prob
    results[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred),
        "ROC-AUC": roc_auc_score(y_test, y_prob) if (y_prob is not None and len(np.unique(y_test)) > 1) else None
    }

## Exercise: Modify the ANN structure
Try changing the number of layers, neurons, activation functions, or optimizer settings

In [ ]:
# Neural Network (ANN) with three layers
ann = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # First hidden layer with 64 neurons
    Dense(32, activation='relu'),  # Second hidden layer with 32 neurons
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])
ann.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
ann.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=0)  # 50 epochs, batch size 32

In [ ]:
# Predictions for ANN
y_pred_ann = (ann.predict(X_test) > 0.5).astype(int).flatten()
y_prob_ann = ann.predict(X_test)
results["ANN"] = {
    "Accuracy": accuracy_score(y_test, y_pred_ann),
    "Precision": precision_score(y_test, y_pred_ann),
    "Recall": recall_score(y_test, y_pred_ann),
    "F1 Score": f1_score(y_test, y_pred_ann),
    "ROC-AUC": roc_auc_score(y_test, y_prob_ann)
}

In [ ]:
# Transformer-Based Model (TabNet) without the verbose parameter
tabnet = TabNetClassifier()
tabnet.fit(X_train, y_train, eval_set=[(X_test, y_test)], max_epochs=50)

In [ ]:
# Predictions for TabNet
y_pred_tabnet = tabnet.predict(X_test)
y_prob_tabnet = tabnet.predict_proba(X_test)[:,1]
results["TabNet"] = {
    "Accuracy": accuracy_score(y_test, y_pred_tabnet),
    "Precision": precision_score(y_test, y_pred_tabnet),
    "Recall": recall_score(y_test, y_pred_tabnet),
    "F1 Score": f1_score(y_test, y_pred_tabnet),
    "ROC-AUC": roc_auc_score(y_test, y_prob_tabnet)
}

In [ ]:
# Plot ROC curves
plt.figure(figsize=(8, 6))
plotted = False  # To check if at least one valid plot exists

for name, y_prob in y_prob_dict.items():
    if y_prob is not None and len(y_prob) > 0:
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc_score(y_test, y_prob):.2f})")
        plotted = True

if plotted:
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()
else:
    print("No valid probability predictions found. Ensure models support `predict_proba()`.")

### In the above chart, the Random Forest has the best ROC Curve.

In [ ]:
# Display feature importance for XGBoost
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
xgb_model.fit(X_train, y_train)
plt.figure(figsize=(8, 6))
plt.barh(X.columns, xgb_model.feature_importances_)
plt.xlabel('Feature Importance')
plt.ylabel('Feature Name')
plt.title('XGBoost Feature Importance')
plt.show()

### Here, we can calculate the impact of features on the XGBoost model performance:

In [ ]:
# SHAP Explainability
explainer = shap.Explainer(XGBClassifier().fit(X_train, y_train))
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test)  # Feature importance visualization

In [ ]:
# Display results as a DataFrame
results_df = pd.DataFrame(results).T
print("Model Performance Comparison:\n")
print(results_df)

### Usually, XGBoost and TabNet achieve the highest performances; however, here, they achieved weak results in comparison to e.g. RandomForest.

In [ ]:
# If using Jupyter Notebook, display it as a table
import IPython.display as display
display.display(results_df)

### As visible, no method is perfect for these datasets. Try to change some hyperparameters and experiment to collect better results.

Try to change some XGBoost parameters and analyze how it influences the performance of this model on a chosen dataset in comparison to previous results.

In [ ]:
# Exercise: Experiment with XGBoost Hyperparameters
def tune_xgboost():
    param_grid = {
        'n_estimators': [50, 100, 200],  # Experiment with the number of boosting rounds
        'learning_rate': [0.01, 0.1, 0.2],  # Adjust the step size
        'max_depth': [3, 5, 7],  # Increase model complexity
        'subsample': [0.8, 1.0],  # Reduce overfitting by sampling data
        'colsample_bytree': [0.8, 1.0]  # Feature selection at each boosting step
    }
    grid_search = GridSearchCV(XGBClassifier(), param_grid, cv=5, scoring='roc_auc', verbose=1, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    print("Best XGBoost Hyperparameters:", grid_search.best_params_)
    return grid_search.best_estimator_

I have added an exercise for students to experiment with XGBoost hyperparameters using GridSearchCV. The exercise allows students to test different values for:
<ul>
<li>n_estimators</li>
<li>learning_rate</li>
<li>max_depth</li>
<li>subsample</li>
<li>colsample_bytree</li>
</ul>

In [ ]:
# Now, you can uncomment the following line to experiment with hyperparameter tuning using GridSearchCV:
# best_xgb = tune_xgboost()

In [ ]:
# Display results as a DataFrame
results_df = pd.DataFrame(results).T
print("Model Performance Comparison:")
print(results_df)

# Display results in Jupyter Notebook
import IPython.display as display
display.display(results_df)

## Exercises to better understand and work with biomedical data
1. Experiment with hyperparameters (e.g., n_estimators, max_depth, learning_rate, kernel types).
2. Modify the ANN structure (changing the number of layers, neurons, activation functions, or optimizer settings).
3. Try different biomedical datasets (e.g., Heart Disease, Breast Cancer, Stroke Prediction from Kaggle/UCI ML Repository) - some of them are available in the above code - prepared for your experiments.
4. Compare and interpret results.

## Final notice

Use the gained experience to gradually construct your solution on a different dataset to complete the assignment with tabular data.